In [28]:
import os
import random
import re
import string

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [2]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yuvraj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yuvraj\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yuvraj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [30]:
def load_lines(file_path):
    lines = {}
    with open(file_path, "r", encoding="iso-8859-1") as f:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 5:
                # Only process lines with 5 parts
                line_id = parts[0]
                text = parts[4]
                lines[line_id] = text
    return lines

def load_conversations(file_path, lines):
    conversations = {}
    with open(file_path, "r", encoding="iso-8859-1") as f:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 4:
                # Only process conversations with 4 parts
                try:
                    conv_id = parts[0]
                    line_ids = parts[3][1:-1].replace("'", "").replace(" ", "").split(",")
                    conversations[conv_id] = [lines[line_id] for line_id in line_ids]
                except KeyError:
                    # Skip this conversation if one of the lines is missing
                    continue
                
    return conversations

# Load the dataset
corpus_path = os.path.join("data", "cornell_movie_dialog")
lines_path = os.path.join(corpus_path, "movie_lines.txt")
convs_path = os.path.join(corpus_path, "movie_conversations.txt")

lines = load_lines(lines_path)
conversations = load_conversations(convs_path, lines)

qa_pairs = create_qa_pairs(lines, conversations)


# Create a dictionary that maps each message to its response
def create_qa_pairs(lines, conversations):
    qa_pairs = {}
    for conv_lines in conversations.values():
        for i in range(len(conv_lines) - 1):
            input_line = conv_lines[i].strip()
            target_line = conv_lines[i+1].strip()
            qa_pairs[input_line] = target_line
    return qa_pairs

In [39]:
# Preprocess the text
def preprocess(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered = [word for word in tokens if word not in stop_words]
    # Stem the words
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(word) for word in filtered]
    # i Remove short questions
#     if len(stemmed) < 3:
#         return None
    # Join the words back into a string
    return " ".join(stemmed)

# Load the dataset and preprocess the text
# lines = load_lines()
# conversations = load_conversations()
# # print(conversations)
# qa_pairs = create_qa_pairs(lines, conversations)
preprocessed = [preprocess(q) for q in qa_pairs.keys() if preprocess(q)]

# Train the chatbot
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(preprocessed)

def respond(user_input):
    input_tfidf = vectorizer.transform([preprocess(user_input)])
    similarities = cosine_similarity(input_tfidf, tfidf).flatten()
    best_match_index = similarities.argsort()[-1]
    if similarities[best_match_index] == 0:
        return "I'm sorry, I don't understand."
    else:
        response = list(qa_pairs.values())[best_match_index]
        return response

In [41]:
# Main loop
while True:
    user_input = input("> ")
    response = respond(user_input)
    print("Chatbot:", response)

# while True:
#     # Get input from user
#     input_str = input("You: ")
    
    
    
#     # Preprocess the input string
#     input_str = preprocess(input_str)
#     print(input_str)
#     if input_str is None:
#         print("Sorry, I didn't understand that. Please try again.")
#         continue
    
#     # Generate response using the trained model
#     input_vec = vectorizer.transform([input_str])
#     cosine_similarities = cosine_similarity(input_vec, tfidf)
#     max_similarity_index = cosine_similarities.argmax()
#     response = qa_pairs_list[max_similarity_index][1]
    
#     # Print response
#     print("Chatbot:", response)

> HI
Chatbot: Hi, is Vada home?
> can you help me?
Chatbot: You've got to help them.
> ok bro
Chatbot: All right, Scooby, let's not beat around the bush. With your attitude, you're not going to get in anywhere.


KeyboardInterrupt: Interrupted by user